# Gemma-2-2b-it 모델 파인튜닝

## 0. 학습 데이터 확인

In [1]:
import pandas as pd

# ✅ CSV 파일 로드
csv_path = "extracted_documents.csv"
df = pd.read_csv(csv_path, encoding="utf-8")

# ✅ 데이터 확인
print("📌 데이터셋 컬럼명:", df.columns)
print("📌 데이터 샘플:\n", df.head())

# ✅ 컬럼명 변경 (필요 시)
df = df.rename(columns={"original_text": "input_text", "summary_text": "target_text"})

# ✅ 저장 (변경된 파일을 활용할 경우)
df.to_csv("extracted_documents_updated.csv", index=False, encoding="utf-8-sig")


📌 데이터셋 컬럼명: Index(['original_text', 'summary_text'], dtype='object')
📌 데이터 샘플:
                                        original_text  \
0  원고가 소속회사의 노동조합에서 분규가 발생하자 노조활동을 구실로 정상적인 근무를 해...   
1  수출입업체인 원고가 의류제품을 제조ㆍ수출함에 있어 같은 그룹내 종합무역상사인 소외 ...   
2  가등기담보권자가 제소전 화해조항에 따라 자기 명의로 소유권이전의 본등기를 경료한 후...   
3  가. 부가가치세법 제22조 제3항 단서에 제1호와 제2호가 동시에 해당한다는 뜻은 ...   
4  소득세법 제116조 제1항의 규정에 의하면 정부는 과세표준확정신고를 하여야 할 자에...   

                                        summary_text  
0  원고가  주동하여 회사업무능률을 저해하고 회사업무상의 지휘명령에 위반하였다면 이에 ...  
1  수출입업체인 원고가 의류제품을 제조ㆍ수출함에 있어 소외 회사의 직수출실적을 지원하기...  
2  가등기담보권자가 제소전 화해조항에 의해 자기 명의로 소유권이전의 본등기를 경료하고 ...  
3  부가가치세법 제22조 제3항 단서에 제1호와 제2호가 동시에 해당한다는 의미는 제1...  
4  소득세법 제116조 제1항에 따르면 정부는 과세표준확정신고를 해야 할 자에 대해 당...  


## 1. 모델 및 토크나이저 로드

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
from main import access_token  # ✅ main.py에서 Access Token 가져오기

def load_model_and_tokenizer(model_name="google/gemma-2-2b-it", quantized=True):
    """
    Hugging Face에서 Gemma 모델과 토크나이저를 로드하는 함수.
    - QLoRA 적용 (4-bit 양자화)
    - main.py에서 Access Token을 가져와 인증
    """
    if not access_token:
        raise ValueError("❌ Hugging Face Access Token이 없습니다. 환경 변수를 확인하세요.")

    tokenizer = AutoTokenizer.from_pretrained(model_name, token=access_token)

    if quantized:
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.float16,  # ✅ FP16 연산 사용
            bnb_4bit_quant_type="nf4",  # ✅ QLoRA 최적화된 nf4 양자화 적용
            bnb_4bit_use_double_quant=True  # ✅ 이중 양자화 적용
        )
        device_map = {"": "cuda:1"} if torch.cuda.is_available() else "cpu"  # ✅ GPU 설정

        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            quantization_config=bnb_config,
            device_map=device_map,
            token=access_token  # ✅ 인증 추가
        )
    else:
        model = AutoModelForCausalLM.from_pretrained(model_name, token=access_token)

    return model, tokenizer


## 2. 데이터셋 불러오기

In [3]:
from datasets import Dataset

def prepare_dataset(csv_path, encoding="utf-8"):
    """
    CSV 파일을 불러와서 데이터셋을 구축합니다.
    'input_text'와 'target_text' 컬럼을 Hugging Face Dataset 형식으로 변환합니다.
    """
    df = pd.read_csv(csv_path, encoding=encoding)

    # ✅ 컬럼명 확인 후 변환
    if "original_text" in df.columns and "summary_text" in df.columns:
        df = df.rename(columns={"original_text": "input_text", "summary_text": "target_text"})
    elif "input_text" not in df.columns or "target_text" not in df.columns:
        raise ValueError("CSV 파일에 'input_text' 또는 'target_text' 컬럼이 없습니다. 컬럼명을 확인하세요!")

    # ✅ Pandas DataFrame → Hugging Face Dataset 변환
    dataset = Dataset.from_pandas(df)
    return dataset

# ✅ 사용 예시
csv_path = "extracted_documents_updated.csv"
raw_dataset = prepare_dataset(csv_path, encoding="utf-8")
print("✅ 준비된 데이터셋 예시:", raw_dataset[0])


✅ 준비된 데이터셋 예시: {'input_text': '원고가 소속회사의 노동조합에서 분규가 발생하자 노조활동을 구실로 정상적인 근무를 해태하고, 노조조합장이 사임한 경우, 노동조합규약에 동 조합장의 직무를 대행할 자를 규정해 두고 있음에도 원고 자신이 주동하여 노조자치수습대책위원회를 구성하여 그 위원장으로 피선되어 근무시간중에도 노조활동을 벌여 운수업체인 소속회사의 업무에 지장을 초래하고 종업원들에게도 나쁜 영향을 끼쳐 소속회사가 취업규칙을 위반하고 고의로 회사업무능률을 저해하였으며 회사업무상의 지휘명령에 위반하였음을 이유로 원고를 징계해고 하였다면, 이는 원고의 노동조합 활동과는 관계없이 회사취업규칙에 의하여 사내질서를 유지하기 위한 사용자 고유의 징계권에 기하여 이루어진 정당한 징계권의 행사로 보아야 한다.', 'target_text': '원고가  주동하여 회사업무능률을 저해하고 회사업무상의 지휘명령에 위반하였다면 이에 따른 징계해고는 사내질서를 유지하기 위한 사용자 고유의 정당한 징계권의 행사로 보아야 한다.'}


## 3. 데이터 토큰화

In [5]:
def tokenize_dataset(dataset, tokenizer, max_input_length=1024, max_target_length=256):
    """
    데이터셋을 토큰화하여 모델이 학습할 수 있는 형태로 변환합니다.
    """
    if tokenizer is None:
        raise ValueError("❌ tokenizer가 정의되지 않았습니다. `load_model_and_tokenizer`를 호출하여 tokenizer를 먼저 로드하세요.")

    def tokenize_function(example):
        model_inputs = tokenizer(
            example["input_text"],
            max_length=max_input_length,
            truncation=True,
            padding="max_length"
        )
        labels = tokenizer(
            example["target_text"],
            max_length=max_target_length,
            truncation=True,
            padding="max_length"
        )
        model_inputs["labels"] = labels["input_ids"]
        return model_inputs

    tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=dataset.column_names)
    return tokenized_dataset

# ✅ 모델 및 토크나이저 로드
model, tokenizer = load_model_and_tokenizer("google/gemma-2-2b-it")

# ✅ 데이터셋 로드
csv_path = "extracted_documents_updated.csv"
raw_dataset = prepare_dataset(csv_path, encoding="utf-8")

# ✅ 수정된 코드 사용
tokenized_dataset = tokenize_dataset(raw_dataset, tokenizer)
print("✅ 토큰화 완료! 첫 번째 데이터 샘플:", tokenized_dataset[0])


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Map:   0%|          | 0/24329 [00:00<?, ? examples/s]

✅ 토큰화 완료! 첫 번째 데이터 샘플: {'input_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

## 4. Fine_Tuning

In [6]:
import torch
from transformers import TrainingArguments, Trainer, DataCollatorForSeq2Seq
from peft import LoraConfig, get_peft_model

def fine_tune_model(model, tokenizer, tokenized_dataset, output_dir="./fine_tuned_results", epochs=3, batch_size=1):
    """
    LoRA 기반 Fine-Tuning 실행 함수.
    """

    # ✅ LoRA 어댑터 구성
    lora_config = LoraConfig(
        r=16,  # LoRA 랭크 (메모리에 맞게 조정 가능)
        lora_alpha=32,  # LoRA 학습률 계수
        lora_dropout=0.05,  # 드롭아웃 적용
        bias="none",
        task_type="CAUSAL_LM"  # 캐주얼 언어 모델링 (Gemma 모델에 맞춤)
    )

    # ✅ LoRA 적용
    model = get_peft_model(model, lora_config)
    
    # ✅ 데이터셋 분할: 학습(train) & 검증(eval)
    split_dataset = tokenized_dataset.train_test_split(test_size=0.1, seed=42)
    train_dataset = split_dataset["train"]
    eval_dataset = split_dataset["test"]
    
    # ✅ 데이터 콜레이터: 패딩 자동 처리
    data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

    # ✅ 학습 파라미터 설정
    training_args = TrainingArguments(
        output_dir=output_dir,
        evaluation_strategy="epoch",
        learning_rate=3e-4,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=epochs,
        weight_decay=0.01,
        fp16=True,  # 16-bit 연산 사용
        save_total_limit=2,
        logging_steps=50,
        seed=42
    )

    # ✅ Trainer 객체 생성
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
        data_collator=data_collator
    )

    # ✅ Fine-Tuning 실행
    trainer.train()

    # ✅ 학습 완료된 모델 저장
    trainer.save_model(output_dir)

# ✅ 사용 예시
# fine_tune_model(model, tokenizer, tokenized_dataset, output_dir="./fine_tuned_results")


## 5. 메인 함수 실행 코드

In [7]:
import os
import torch

def main():
    # ✅ 환경 변수 설정 (메모리 단편화 완화)
    os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

    # ✅ GPU 설정
    device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
    print("Using device:", device)

    # ✅ 모델 및 토크나이저 로드
    model, tokenizer = load_model_and_tokenizer("google/gemma-2-2b-it", quantized=True)

    # ✅ 데이터 로드 및 토큰화
    csv_path = "extracted_documents_updated.csv"
    raw_dataset = prepare_dataset(csv_path, encoding="utf-8")
    tokenized_dataset = tokenize_dataset(raw_dataset, tokenizer)

    # ✅ Fine-Tuning 실행
    fine_tune_model(model, tokenizer, tokenized_dataset, output_dir="./fine_tuned_results")

if __name__ == "__main__":
    main()


Using device: cuda:1


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Map:   0%|          | 0/24329 [00:00<?, ? examples/s]

/home/wanted-1/miniconda3/envs/dl_project/lib/python3.9/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/tmp/ipykernel_2320502/28924360.py:46: FutureWarning: `tokenizer` is deprecated and will be rem

[2025-03-02 09:41:39,923] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cuda (auto detect)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

ValueError: You can't train a model that has been loaded in 8-bit or 4-bit precision on a different device than the one you're training on. Make sure you loaded the model on the correct device using for example `device_map={'':torch.cuda.current_device()}` or `device_map={'':torch.xpu.current_device()}`